In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from datetime import timedelta
from sklearn.grid_search import GridSearchCV

import sys
sys.path.append('/Users/drewrice/Desktop/Galvanize/Github/Galvanize-Final-Project/code')
from return_low_no_suit_score import return_low_no_suit_score

# Imports and feature selection

In [127]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds_full = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')
policy = pd.read_csv('/Users/drewrice/Desktop/capson_csv/PolicyInfo.csv')

In [128]:
# drop non-essential columns
# drop duplicates
docs.drop(['Middle Name','Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [129]:
# grab essential columns for building classifier
indem = claims[['Indemnity_Paid','ProviderId']]

In [130]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [131]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [132]:
# def build_classifier_label(item):
#     if item == 0:
#         return 'suit, no indem'
#     if item != 0:
#         return 'suit, indem'

def build_classifier_label(item):
    if item == 0:
        return 1
    if item != 0:
        return 2

In [133]:
indem['indem_label'] = indem['Indemnity_Paid'].apply(lambda item: build_classifier_label(item))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Join docs and indemnity tables, fill NaNs with 'no suit'

In [134]:
joined = pd.merge(docs, indem[['ProviderId', 'indem_label']], how='left', on='ProviderId')
joined.drop_duplicates(inplace=True)

In [135]:
# finish indemnity classifier
# find NaNs in indem_label column, replace with 'no suit'
def indem_label_no_suit(x):
    if type(x) == np.float_: 
        if np.isnan(x): 
#             return 'no suit'
            return 0
        else:
            return x

In [136]:
joined.indem_label = joined.indem_label.apply(lambda item: indem_label_no_suit(item))

# Claims data
#### building classifiers for BoardCredit and ClaimsBand, then merging with joined

In [137]:
def build_classifier(item):
    if item == 0:
        return 0
    if item != 0:
        return 1

In [138]:
# resave debs_and_creds with just the desired columns
# NOTE: typo 'RecordKepping'
debs_and_creds = debs_and_creds_full[['ProviderId','BoardCredit','ClaimsBand','Procedures','JCAHO','RecordKepping']]

# lower Provider Id
debs_and_creds['ProviderId'] = debs_and_creds['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())
debs_and_creds.tail()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ProviderId,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping
8458,19d9c4e6-e820-41c4-b398-134e6c0ba410,-55,-220,0,0,0
8459,6398eeea-10af-448f-bd10-364be1d0c90c,0,-8951,0,-2984,0
8460,d54c0b59-08e5-4ed7-ba5a-413b2016b269,-1366,0,0,-1366,-2733
8461,3eacedf6-9173-4a64-89c7-ba8b6e150850,-88,0,0,0,0
8462,ea83f592-67dd-46d5-b15c-9d0738f81c93,0,-258,0,0,0


In [139]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()
# reset index required
debs_and_creds.reset_index(inplace=True)

In [140]:
# use build_classifier to create binary column from BoardCredit & ClaimsBand
debs_and_creds['BoardCredit'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['ClaimsBand'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))
debs_and_creds['Procedures'] = debs_and_creds['Procedures'].apply(lambda item: build_classifier(item))
debs_and_creds['JCAHO'] = debs_and_creds['JCAHO'].apply(lambda item: build_classifier(item))
debs_and_creds['RecordKepping'] = debs_and_creds['RecordKepping'].apply(lambda item: build_classifier(item))

# Build 'joined'

In [203]:
joined = pd.merge(joined, debs_and_creds, how='left', on='ProviderId')

# EDA

In [204]:
# print num of doctors in each bucket, using the 
print 'Doctors wih no suits: ', joined[joined['indem_label'] == 0].sort(['Last Name']).shape[0]
print 'Doctors with suits, no indemnity paid: ', joined[joined['indem_label'] == 1].sort(['Last Name']).shape[0]
print 'Doctors with suits, indemnity paid: ', joined[joined['indem_label'] == 2].sort(['Last Name']).shape[0]

Doctors wih no suits:  2533
Doctors with suits, no indemnity paid:  246
Doctors with suits, indemnity paid:  34


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# Feature engineering

### ~ build age column ~

In [205]:
# convert DoB to datetime object
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [206]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [207]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [208]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [209]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [210]:
# function to convert years into days
def to_years(dt):
    if dt:
        dt = (dt.days) / 365.25
    return dt

In [211]:
# apply to_years function to age column
joined['age'] = (joined['age'].apply(lambda dt: to_years(dt))).round()

In [212]:
# refill Date of Birth with NaNs
joined['Date of Birth'].replace(datetime(2060, 1, 1, 0, 0), np.nan, inplace=True)

In [213]:
def refill_age(x):
    if x < 0:
        x = np.nan
    return x

In [214]:
# refill age with nan's for docs with missing birthdate
joined['age'] = (joined['age'].apply(lambda dt: refill_age(dt))).round()

In [215]:
joined.corr()

,indem_label,BoardCredit_x,ClaimsBand_x,Procedures_x,JCAHO_x,RecordKepping_x,age,BoardCredit_y,ClaimsBand_y,Procedures_y,JCAHO_y,RecordKepping_y,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping
indem_label,1.000000,0.132062,0.106334,0.155475,0.190584,0.049263,0.104879,0.132062,0.106334,0.155475,0.190584,0.049263,0.132062,0.106334,0.155475,0.190584,0.049263
BoardCredit_x,0.132062,1.000000,0.511192,0.041787,0.422139,0.166218,0.103663,1.000000,0.511192,0.041787,0.422139,0.166218,1.000000,0.511192,0.041787,0.422139,0.166218
ClaimsBand_x,0.106334,0.511192,1.000000,0.082723,0.434187,0.219986,0.347871,0.511192,1.000000,0.082723,0.434187,0.219986,0.511192,1.000000,0.082723,0.434187,0.219986
Procedures_x,0.155475,0.041787,0.082723,1.000000,0.008991,-0.028698,0.049746,0.041787,0.082723,1.000000,0.008991,-0.028698,0.041787,0.082723,1.000000,0.008991,-0.028698
JCAHO_x,0.190584,0.422139,0.434187,0.008991,1.000000,0.255825,0.068109,0.422139,0.434187,0.008991,1.000000,0.255825,0.422139,0.434187,0.008991,1.000000,0.255825
RecordKepping_x,0.049263,0.166218,0.219986,-0.028698,0.255825,1.000000,-0.000548,0.166218,0.219986,-0.028698,0.255825,1.000000,0.166218,0.219986,-0.028698,0.255825,1.000000
age,0.104879,0.103663,0.347871,0.049746,0.068109,-0.000548,1.000000,0.103663,0.347871,0.049746,0.068109,-0.000548,0.103663,0.347871,0.049746,0.068109,-0.000548
BoardCredit_y,0.132062,1.000000,0.511192,0.041787,0.422139,0.166218,0.103663,1.000000,0.511192,0.041787,0.422139,0.166218,1.000000,0.511192,0.041787,0.422139,0.166218
ClaimsBand_y,0.106334,0.511192,1.000000,0.082723,0.434187,0.219986,0.347871,0.511192,1.000000,0.082723,0.434187,0.219986,0.511192,1.000000,0.082723,0.434187,0.219986
Procedures_y,0.155475,0.041787,0.082723,1.000000,0.008991,-0.028698,0.049746,0.041787,0.082723,1.000000,0.008991,-0.028698,0.041787,0.082723,1.000000,0.008991,-0.028698


In [216]:
joined.dropna(how='any', inplace=True)
joined = joined.query('State != "nan"')
joined.shape

(2813, 25)

# Basic model

In [217]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [218]:
# drop rows with any NaN values
# not a long-term solution, but a quick and dirty model
no_suit = joined[joined.indem_label == 0]
suit_no_indem = joined[joined.indem_label == 1]
suit_indem = joined[joined.indem_label == 2]

no_suit.dropna(how='any', inplace=True)
suit_no_indem.dropna(how='any', inplace=True)
suit_indem.dropna(how='any', inplace=True)

# print totals after NaN drop
print no_suit.shape[0], suit_indem.shape[0], suit_no_indem.shape[0]

# !
# change values to 1 for binary classification
# suit_indem['indem_label'] = 1

# random sample from the three categories
no_suit = no_suit.sample(600)
suit_no_indem = suit_no_indem.sample(246)
suit_indem = suit_indem.sample(34)

2533 34 246


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [219]:
# combine random sample dataframes into concat_df
concat_df = pd.concat([no_suit, suit_indem, suit_no_indem], axis=0)

In [220]:
# build label and feature dataframes
y = concat_df.pop('indem_label')
to_model = concat_df[['BoardCredit','ClaimsBand','Procedures','JCAHO','age','RecordKepping']]
state = concat_df.pop('State')
dummies = pd.get_dummies(state)
to_model = pd.concat([to_model,dummies],axis=1)

In [221]:
# train / test split with random state
X_train, X_test, y_train, y_test = train_test_split(to_model, y, test_size=0.10)

In [222]:
model = RandomForestClassifier(n_estimators=95,oob_score=True)

In [223]:
# fit, validate with OOB score
model.fit(X_train,y_train)
model.oob_score_

0.64393939393939392

In [224]:
preds = model.predict(X_test)
model.score(X_test, y_test)

0.61363636363636365

In [225]:
confusion_matrix(y_test, preds)

array([[48, 13,  2],
       [16,  6,  1],
       [ 1,  1,  0]])

In [226]:
# grab indices from return_low_no_suit_score FUNC
low_no_suit_scores = return_low_no_suit_score(model,X_test)
indices = [x[0] for x in low_no_suit_scores]

# make y_test a list of values, from DataFrame
y_actual_list = y_test.values.tolist()
# take indices from return_low_no_suit_score FUNC
y_actual_list_appropriate_indices = [y_actual_list[x] for x in indices]

zip(low_no_suit_scores,y_actual_list_appropriate_indices)

[([3, 0.1842, 0.5264, 0.2894], 1.0),
 ([5, 0.4842, 0.5158, 0.0], 1.0),
 ([6, 0.6467, 0.3533, 0.0], 1.0),
 ([7, 0.4614, 0.4916, 0.047], 0.0),
 ([8, 0.6314, 0.3686, 0.0], 0.0),
 ([11, 0.681, 0.319, 0.0], 0.0),
 ([14, 0.5204, 0.4796, 0.0], 1.0),
 ([15, 0.2184, 0.3395, 0.4421], 0.0),
 ([18, 0.78, 0.22, 0.0], 1.0),
 ([22, 0.113, 0.5265, 0.3605], 0.0),
 ([23, 0.0919, 0.4984, 0.4096], 0.0),
 ([27, 0.1789, 0.8211, 0.0], 0.0),
 ([28, 0.1307, 0.8693, 0.0], 0.0),
 ([34, 0.6842, 0.2842, 0.0316], 0.0),
 ([35, 0.0911, 0.494, 0.4149], 0.0),
 ([36, 0.0632, 0.9167, 0.0202], 0.0),
 ([37, 0.4579, 0.5421, 0.0], 0.0),
 ([38, 0.2921, 0.0868, 0.6211], 0.0),
 ([39, 0.0175, 0.9719, 0.0105], 1.0),
 ([41, 0.0632, 0.9167, 0.0202], 1.0),
 ([42, 0.5368, 0.4491, 0.014], 1.0),
 ([43, 0.4846, 0.5083, 0.007], 0.0),
 ([44, 0.7368, 0.2211, 0.0421], 0.0),
 ([45, 0.7149, 0.243, 0.0421], 0.0),
 ([49, 0.6087, 0.3913, 0.0], 1.0),
 ([51, 0.3614, 0.6175, 0.0211], 1.0),
 ([53, 0.704, 0.296, 0.0], 0.0),
 ([55, 0.6842, 0.2421, 0.0

#### Gridsearching! Hooray for premature optimization

In [54]:
n_est_range = [32,64,80,85,90,95,100,120]
m_range = [2,3,4,5,6,8,16,32]
lrgs = GridSearchCV(estimator=model, param_grid=dict(n_estimators=n_est_range, max_depth=m_range), n_jobs=1)
lrgs.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=85, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [32, 64, 80, 85, 90, 95, 100, 120], 'max_depth': [2, 3, 4, 5, 6, 8, 16, 32]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [55]:
print lrgs.best_score_
print lrgs.best_estimator_

0.7
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=95, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)


(2813, 15)

# Build CSV of docs with location and indemnity label

,ProviderId,Last Name,First Name,Gender,Date of Birth,City,State,County,indem_label,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping,age
1,9977d6b0-04eb-4310-be27-8470257da33e,Abalos,Jennifer,Unknown,1972-10-01,Plano,TX,DALLAS,0.0,0.0,0.0,0.0,0.0,0.0,44.0
2,5ed9f93e-5bf2-4990-bd31-fb0754ae3aec,Abbasi,Noma,Unknown,1971-11-01,Sugar Land,TX,FORT BEND,0.0,1.0,1.0,0.0,0.0,0.0,45.0
3,949a1121-3f0f-41f2-a6df-b4acc1b64f9a,Abben,Richard,Male,1952-10-11,Houma,LA,TERREBONNE,0.0,1.0,1.0,0.0,1.0,0.0,64.0
6,e35196bd-a7a2-4f07-95e9-4a47775e0ac8,Abdul Rashid,Nik,Unknown,1970-03-30,Las Vegas,NV,CLARK,0.0,1.0,1.0,0.0,1.0,0.0,46.0
7,f9f98580-e6db-4428-8fab-6563b8574c22,Abdulla,Amina,Female,1946-07-30,Houston,TX,HARRIS,0.0,0.0,1.0,0.0,0.0,0.0,70.0
8,4590a5c6-a9d8-4b5e-946f-955f7373eade,Aboubaker,Kaiss,Unknown,1973-08-04,Dallas,TX,DALLAS,0.0,0.0,0.0,0.0,0.0,0.0,43.0
9,805a9da9-be65-45fc-9e7e-77cbe4a37036,Aboyeji,Olufemi,Male,1978-09-26,Schererville,IN,LAKE,0.0,1.0,1.0,0.0,1.0,0.0,38.0
11,357475c2-2193-4459-994c-46a20c22c225,Abraham,Jolly,Female,1982-06-24,Katy,TX,HARRIS,0.0,0.0,0.0,0.0,0.0,0.0,34.0
12,11192bc0-363f-4a62-afad-33a878f930eb,Abreha,Amanuel,Male,1962-08-01,Houston,TX,HARRIS,0.0,0.0,1.0,0.0,1.0,0.0,54.0
13,e05bae45-095a-47ee-9775-fe4f95d33af5,Abu-Deiab,Mohammed,Male,1984-03-18,Edinburg,TX,HIDALGO,0.0,0.0,0.0,0.0,0.0,0.0,32.0
